## Import

In [ ]:
# pip install scikit-learn
# pip install fastparquet

In [1]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

## csv to parquet

In [2]:
# def seed_everything(seed):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
# seed_everything(42) # Seed 고정

# def csv_to_parquet(csv_path, save_name):
#     df = pd.read_csv(csv_path)
#     df.to_parquet(f'./{save_name}.parquet')
#     del df
#     gc.collect()
#     print(save_name, 'Done.')
    
# csv_to_parquet('./open/train.csv', 'train')
# csv_to_parquet('./open/test.csv', 'test')

## Data Load


In [3]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
sample_submission = pd.read_csv('./open/sample_submission.csv', index_col = 0)


In [4]:
train.head(7)

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,None
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,None
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,None
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,None
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,None
5,TRAIN_000005,4,13,1545.0,NaN,0,0,EWR,11618,None,DCA,11278,Virginia,199.0,Republic Airlines,UA,20452.0,N657RW,Not_Delayed
6,TRAIN_000006,1,20,1742.0,1903.0,0,0,EWR,11618,New Jersey,BOS,10721,Massachusetts,200.0,United Air Lines Inc.,UA,NaN,N66825,Not_Delayed


## Data Pre-Processing


In [14]:
# print(train.loc[train['Origin_Airport']=="OKC"] & train.loc[train['Destination_Airport']=="HOU"])
train.loc[(train['Origin_Airport'] == "OKC") & (train['Destination_Airport']=="HOU"),]

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,None
827,TRAIN_000827,7,16,550.0,715.0,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N733SA,None
1452,TRAIN_001452,1,13,1405.0,1535.0,0,0,OKC,13851,Oklahoma,HOU,12191,None,419.0,Southwest Airlines Co.,WN,19393.0,N204WN,None
4015,TRAIN_004015,5,17,1050.0,1215.0,0,0,OKC,13851,Oklahoma,HOU,12191,None,419.0,Southwest Airlines Co.,WN,19393.0,N246LV,None
9325,TRAIN_009325,8,14,600.0,725.0,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7706A,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991185,TRAIN_991185,1,10,1915.0,2045.0,0,0,OKC,13851,None,HOU,12191,Texas,419.0,None,WN,19393.0,N8684F,Not_Delayed
991368,TRAIN_991368,8,14,1450.0,1610.0,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N8621A,None
993122,TRAIN_993122,1,14,600.0,730.0,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N765SW,None
995469,TRAIN_995469,1,29,1940.0,2105.0,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,None,19393.0,N243WN,None


In [ ]:
#레이블(Delay)을 제외한 결측값이 존재하는 변수들을 학습 데이터의 최빈값으로 대체합니다
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)
print('Done.')

In [ ]:
train.head(7)

In [ ]:
#질적 변수들을 수치화합니다
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])
print('Done.')

In [ ]:
train.head(7)

In [ ]:
#레이블이 없는 데이터들을 제거합니다
train = train.dropna()

In [ ]:
train

In [ ]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

In [ ]:
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

## Classification Model Fit


In [ ]:
clf = RandomForestClassifier()
clf.fit(train_x, train_y)